In [6]:
with open('data/airbus_report.xhtml') as f:
    report = f.read()

report[:1000]

'<?xml version="1.0" encoding="UTF-8"?><html xmlns="http://www.w3.org/1999/xhtml" xmlns:link="http://www.xbrl.org/2003/linkbase" xmlns:xbrli="http://www.xbrl.org/2003/instance" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xbrldi="http://xbrl.org/2006/xbrldi" xmlns:iso4217="http://www.xbrl.org/2003/iso4217" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:utr="http://www.xbrl.org/2009/utr" xmlns:num="http://www.xbrl.org/dtr/type/numeric" xmlns:nonnum="http://www.xbrl.org/dtr/type/non-numeric" xmlns:xbrldt="http://xbrl.org/2005/xbrldt" xmlns:ref="http://www.xbrl.org/2006/ref" xmlns:ix="http://www.xbrl.org/2013/inlineXBRL" xmlns:ixt="http://www.xbrl.org/inlineXBRL/transformation/2020-02-12" xmlns:airbus="http://www.airbus.com" xml:lang="en" xmlns:esef_cor="https://www.esma.europa.eu/taxonomy/2022-03-24/esef_cor" xmlns:ifrs-full="https://xbrl.ifrs.org/taxonomy/2022-03-24/ifrs-full"><head><title></title><style type="text/css">#text

In [87]:
from html.parser import HTMLParser
from collections import Counter, defaultdict

class ReportParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.tags = defaultdict(list)
        self.xbrlTags = []
        self.contexts = []
        self.context = []

    def get_current_context(self):
        def process(item):
            res = item['tag']
            if 'class' in item.keys():
                res += f' {item['class']}'
            return res
        
        return " >> ".join([process(item) for item in self.context])
        
    def handle_starttag(self, tag, attr):
        attr = dict(attr)
        curr = {
            'tag': tag, 
        }
        if tag not in ['html', 'body']:
            curr['attr'] = attr
            curr['class'] = attr.get('class', '')
            
        self.context.append(curr)
        
        if tag.startswith('ix') or tag.startswith('xbrl'):
            self.tags[tag].append(attr)
            if tag.startswith('ix'):
                xbrlTag = attr.get('name', '')
                self.xbrlTags.append(xbrlTag)
                cxt = self.get_current_context()
                self.contexts.append(cxt)
            
    def handle_endtag(self, tag):
        self.context.pop(-1)
        pass
        
    def handle_data(self, data):
        if self.context[-1]['tag'] not in ['style']:
            self.context[-1]['data'] = data

parser = ReportParser()
parser.feed(report)
# for context in parser.contexts[:150]:
#     print(context)
#     print('-------------')
# for k, v in parser.tags.items():
#     print(f'{k}: {len(v)}')
#     print(f'{v[0]}\n')
# Counter(parser.xbrlTags)
Counter(parser.contexts)


Counter({'html >> body >> div page cd cbz3 >> div container c2 >> ix:continuation ': 81,
         'html >> body >> div page cd cbz3 >> div container c2 >> ix:continuation  >> span wrapper >> ix:continuation ': 78,
         'html >> body >> div page cd cbz3 >> div container c2 >> ix:continuation  >> span wrapper >> ix:continuation  >> span wrapper >> ix:continuation ': 62,
         'html >> body >> div page cd cbz3 >> div container c2 >> ix:continuation  >> span wrapper >> ix:continuation  >> span wrapper >> ix:continuation  >> span wrapper >> ix:continuation ': 48,
         'html >> body >> div page cd cbz3 >> div container c2 >> ix:continuation  >> span wrapper >> ix:continuation  >> span wrapper >> ix:continuation  >> span wrapper >> ix:continuation  >> span wrapper >> ix:continuation ': 32,
         'html >> body >> div page cd cbz3 >> div container c2 >> ix:continuation  >> span wrapper >> ix:continuation  >> span wrapper >> ix:continuation  >> span wrapper >> ix:continuation  >> s

In [36]:
parser.tags['xbrli:scenario'][0]

[]